In [2]:
# Import lib
# ===========================================================
import csv
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
import random
import time
import collections
import math
import sys
from tqdm import tqdm
from time import sleep

import matplotlib.pyplot as plt
# %matplotlib inline
plt.style.use('fivethirtyeight')

from datascience import *
from scipy import stats

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [17]:
# Initialize useful data
# ===========================================================
# with open('clinvar_conflicting_mapped.csv', 'r') as f:
#     reader = csv.reader(f)
#     temp_rows = list(reader)
df = pd.read_csv('clinvar_conflicting_clean.csv', low_memory=False)
df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']] = df[['CLNVI', 'MC', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 
 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 
 'BAM_EDIT', 'SIFT', 'PolyPhen']].fillna(value="null")
columns_backup = df.columns
attributes = list(df.columns)
attribute_dimension = len(attributes) - 1 # eliminate the CLASS column
all_rows = df.values.tolist()
row_num = len(all_rows)
df = df.sample(n = df.shape[0])
df = df.sort_values(by=['CLASS']).head()

df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
39510,12,133202900,G,A,0.0001,0.00003,0.0000,"MedGen:C0027672,SNOMED_CT:699346009|MedGen:C35...",Hereditary_cancer-predisposing_syndrome|Colore...,NC_000012.11:g.133202900G>A,...,Ctg/Ttg,-1.0,null,null,null,0.86400,11.850,1.219017,NaN,0
44730,15,28202766,G,A,0.0039,0.00951,0.0134,"MedGen:C0078918,Orphanet:ORPHA55,SNOMED_CT:638...",Oculocutaneous_albinism|not_specified,NC_000015.9:g.28202766G>A,...,caC/caT,-1.0,OK,null,null,0.08780,3.856,0.123119,NaN,0
18565,5,112176091,A,T,0.0000,0.00000,0.0000,"MedGen:C0027672,SNOMED_CT:699346009",Hereditary_cancer-predisposing_syndrome,NC_000005.9:g.112176091A>T,...,aaA/aaT,1.0,null,tolerated,benign,0.00386,22.700,3.209726,NaN,0
33129,11,17526256,A,G,0.0534,0.13441,0.0262,MedGen:CN169374,not_specified,NC_000011.9:g.17526256A>G,...,null,-1.0,null,null,null,0.92600,2.599,0.001413,NaN,0
52454,17,33446607,C,A,0.0000,0.00000,0.0000,"MedGen:C0027672,SNOMED_CT:699346009|MedGen:C32...",Hereditary_cancer-predisposing_syndrome|Breast...,NC_000017.10:g.33446607C>A,...,tGc/tTc,-1.0,OK,deleterious,probably_damaging,NaN,28.900,6.253661,-2.0,0


In [5]:
# SMOTE Sampling for imbalanced input data
# ===========================================================
cate_columns = [0, 2, 3, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 29, 30, 31]
X_train = [row[: -1] for row in all_rows]
y_train = [row[-1] for row in all_rows]
smt = SMOTENC(random_state=42, categorical_features=cate_columns)
X_train, y_train = smt.fit_resample(X_train, y_train)
# smt = SMOTE()
# X_train, y_train = smt.fit_sample(X_train, y_train)

/Users/frostace/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:532: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)


TypeError: cannot perform reduce with flexible type

In [76]:
# Overwrite the original all_rows with the re-sampled data
# ===========================================================
all_rows = np.zeros((X_train.shape[0], X_train.shape[1] + 1))
all_rows[:, :X_train.shape[1]] = X_train
all_rows[:, X_train.shape[1]] = y_train
df = pd.DataFrame(all_rows)
df.columns = columns_backup
df = df.sample(n = df.shape[0])
df = df.sample(n = df.shape[0])
df.astype({'CLASS': 'int32'})

all_rows = df.values.tolist()
row_num = len(all_rows)
df.head()
# np.bincount([row[-1] for row in all_rows])

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
63841,0.458333,0.553925,0.578522,0.746725,0.000000,0.000000,0.000000,0.830301,0.630778,0.873504,...,0.766772,0.9375,0.666667,0.8,0.8,0.418084,0.231375,0.290500,0.000000,1.0
15905,0.875000,0.757397,0.685807,0.568441,0.133680,0.336305,0.000000,0.549886,0.898945,0.837499,...,0.484200,0.9375,0.333333,0.8,0.8,0.524408,0.503431,0.443657,0.999747,1.0
14455,0.315805,0.939459,0.306951,0.746725,0.000000,0.000000,0.000000,0.881881,0.758153,0.278621,...,0.671769,0.0625,0.333333,0.8,0.8,0.858692,0.448062,0.165875,0.000000,1.0
8709,0.416667,0.574000,0.764434,0.567686,0.000000,0.000000,0.753234,0.773338,0.354320,0.651470,...,0.671769,0.0625,0.666667,0.8,0.8,0.880178,0.452957,0.169597,0.000000,0.0
18611,0.875000,0.811170,0.578522,0.746725,0.070373,0.991450,0.068040,0.982131,0.569978,0.073372,...,0.671769,0.9375,0.666667,0.8,0.8,0.917530,0.240399,0.217382,0.000000,0.0


In [77]:
# Divide whole dataset into training set and testing set
# ===========================================================
training_percentage = 0.5  # percent of partition of training dataset
training_size = int(row_num * training_percentage)
testing_size = row_num - training_size
training_data = all_rows[: training_size]  # training data should include header row
testing_data = all_rows[training_size: ]   # testing data don't need to include header row
# np.bincount([row[-1] for row in training_data])
# np.bincount([row[-1] for row in testing_data])

# Instruction
1. Select the number of clusters you want to identify in your data 
2. Randomly select k distinct data points 
3. Measure the distance between the 1st point and the k initial clusters 
4. Assign the 1st point to the nearest cluster 
5. Iterate through all points and do step 3 & 4 
6. Calculate the mean of each cluster 
7. Use the calculated mean of each cluster as k new initial data points and restart from 3 
8. Loop until the mean converge 
9. Do Step 1 - 8 for n times, select the best one 

In [78]:

# i have rescaled all data to confirm that each input dimension ranges within [0, 1]

def gen_random_pivots(k, dim):
    return [np.random.rand(1, dim) for _ in range(k)]

def euclidean_distance(pivot, point):
    # point can have a CLASS entry, we don't want to use it, so eliminate it outside of the function
    
    return np.linalg.norm(np.subtract(pivot, point))

def find_nearest_pivot(point):
    winner = np.random.rand(1, attribute_dimension)
    min_dist = float('inf')
    for i in range(len(pivots)):
        pivot = pivots[i][0]
        temp_dist = euclidean_distance(pivot, point[: -1])
        idx = -2
        if temp_dist < min_dist:
            winner, min_dist, idx = pivot, temp_dist, i
    return winner, idx

def cluster_mean_point(cluster):
    mean_point = np.zeros((1, attribute_dimension))
    for point in cluster:
        mean_point = np.add(mean_point, point[: -1])
    mean_point = np.divide(mean_point, len(cluster) + 0.00000001)
    return mean_point

def clusters_acc(clusters):
    counter1, counter2 = collections.Counter([point[-1] for point in clusters[0]]), collections.Counter([point[-1] for point in clusters[1]])
    one_rate1 = counter1.get(1, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    zero_rate1 = counter1.get(0, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    one_rate2 = counter2.get(1, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    zero_rate2 = counter2.get(0, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    dataset_size = sum(counter1.values()) + sum(counter2.values())
    if one_rate1 > one_rate2:
        # counter1 -> label 1, counter2 -> label 0
        return (counter1.get(1, 0) + counter2.get(0, 0)) / dataset_size
    else:
        return (counter1.get(0, 0) + counter2.get(1, 0)) / dataset_size

In [ ]:
# unsupervised clustering
# ==============================================
K = 2
n = 20 # number of try to run
converge_radius = 0.002

print("training_size: %d" % training_size)
winner = []
min_var = float('inf')
max_acc = -float('inf')

for i in range(n):
    
    # randomly select K distinct data points
    pivots = gen_random_pivots(K, attribute_dimension)
    
    while True:
        # clusters = dict() # i think it would be better to instantize this with a dictionary
        clusters = [[] for _ in range(K)] # init K empty clusters
    
        for point in training_data:
            
            # each row of training_data is a high-dimensional point
            _, idx_of_pivot = find_nearest_pivot(point)
            clusters[idx_of_pivot].append(point)
        temp_pivots = [cluster_mean_point(cluster) for cluster in clusters]
        
        # loop until init_pivots converge
        if np.linalg.norm(np.subtract(pivots, temp_pivots)) < converge_radius:
            break
        pivots = list(temp_pivots)
    
    # pick the best clustering with the most difference between each other
    cluster_sizes = [len(cluster) for cluster in clusters]
    temp_var = np.var(cluster_sizes)
    temp_acc = clusters_acc(clusters)
    if temp_acc > max_acc:
        winner = list(clusters)
        max_acc = temp_acc

'''
* TP: Prediction is True + Predicted value is Positive
* FP: Prediction is False + Predicted value is Positive
* TN: Prediction is True + Predicted value is Negative
* FN: Prediction is False + Predicted value is Negative

* Accuracy = $\frac{TP + TN}{TP + FN + FP + TN}$
* Sensitivity (TPR) = $\frac{TP}{TP + FN}$
* Specificity (FPR) = $\frac{TN}{TN + FP}$
'''

# cause we don't really have cutoff points here, so computing TP, .. would be meaningless
# here we only care about Acc
print("winner: ", len(winner[0]), len(winner[1]))
print("training Acc: ", max_acc)
counter1, counter2 = collections.Counter([point[-1] for point in winner[0]]), collections.Counter([point[-1] for point in winner[1]])
print(counter1, counter2)

training_size: 48754


In [ ]:
# testing
# ===================================================================
# pivots
test_clusters = [[] for _ in range(K)]
cutoff = 0.5
for point in testing_data:
    if euclidean_distance(pivots[0], point[: -1]) / euclidean_distance(pivots[0], pivots[1]) < cutoff:
        test_clusters[0].append(point)
    else:
        test_clusters[1].append(point)
print("testing Acc", clusters_acc(test_clusters))
print(len(test_clusters[0]), len(test_clusters[1]))

In [23]:
# Compute TN, TP, FN, FP, etc.
# ===========================================================
ROC = Table(make_array('CUTOFF', 'TN', 'FN', 'FP', 'TP', 'ACC'))
step_size = 0.05
for cutoff in np.arange(0, 1 + step_size, step_size):
    test_clusters = clusters = [[] for _ in range(K)]
    for point in testing_data:
        if euclidean_distance(pivots[0], point[: -1]) / euclidean_distance(pivots[0], pivots[1]) < cutoff:
            test_clusters[0].append(point)
        else:
            test_clusters[1].append(point)
    counter1, counter2 = collections.Counter([point[-1] for point in test_clusters[0]]), collections.Counter([point[-1] for point in test_clusters[1]])
    one_rate1 = counter1.get(1, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    zero_rate1 = counter1.get(0, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    one_rate2 = counter2.get(1, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    zero_rate2 = counter2.get(0, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    dataset_size = sum(counter1.values()) + sum(counter2.values())
    if one_rate1 > one_rate2:
        # counter1 -> label 1, counter2 -> label 0
        TP = counter1.get(1, 0) / sum(counter1.values() + 1)
        FN = counter1.get(0, 0) / sum(counter1.values() + 1)
        FP = counter2.get(1, 0) / sum(counter2.values() + 1)
        TN = counter2.get(0, 0) / sum(counter2.values() + 1)
    else:
        # counter1 -> label 0, counter2 -> label 1
        TP = counter1.get(1, 0) / sum(counter1.values() + 1)
        FN = counter1.get(0, 0) / sum(counter1.values() + 1)
        FP = counter2.get(1, 0) / sum(counter2.values() + 1)
        TN = counter2.get(0, 0) / sum(counter2.values() + 1)
    output = [cutoff, TN, FN, FP, TP]
    acc = (output[1] + output[4]) / sum(output[1:])
    output.append(acc)
    ROC = ROC.with_row(output)
ROC = ROC.with_columns('SENSITIVITY', ROC.apply(lambda TP, FN: TP / (TP + FN + 0.00000001), 'TP', 'FN'))
ROC = ROC.with_columns('FPR', ROC.apply(lambda TN, FP: FP / (TN + FP + 0.00000001), 'TN', 'FP'))

ZeroDivisionError: division by zero

In [20]:
len(test_clusters[0])
len(test_clusters[1])

97508

In [21]:
len(testing_data)

48754

In [34]:
shit = [[1,2,3], [2,3,4], [5,6,7]]
fuck = pd.DataFrame(shit)

In [48]:
fuck.sample(n=3)

,0,1,2
1,2,3,4
2,5,6,7
0,1,2,3


In [57]:
print(np.bincount([row[-1] for row in training_data]))
print(np.bincount([row[-1] for row in testing_data]))

[24245 24509]
[24509 24245]


In [22]:
df.where(df['CLASS'] == 1)

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
39510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
set([row[-1] for row in all_rows])

{0, 1}